In [1]:
# !pip install perceval-quandela

In [2]:
import perceval as pcvl
import numpy as np
from math import comb
from scipy.optimize import minimize
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
import tqdm as tqdm

KeyboardInterrupt: 

In [ ]:
nphotons = 5


In [ ]:
# Differential equation parameters u(x, t) = x^2 + t^2 + xt

def F(U_x,U_t,x,t):       # DE, works with numpy arrays
    return U_x+U_t-3*x-3*t

# Boundary Fs
def B1(U_0t,T):       # u(0, t) = t^2
    return U_0t-T**2
def B2(U_x0,X):       # u(x, 0) = 0
    return U_x0-X**2
def B3(U_00):       # U(0,0) = 0
    return U_00

In [ ]:
# Boundary condition (f(x_0)=f_0)
# x_0 = 0
# f_0 = np.sin(0)

In [ ]:
# Modeling parameters
n_grid = 10    # number of grid points of the discretized differential equation
range_min = 0  # minimum of the interval on which we wish to approximate our function
range_max = 0.5  # maximum of the interval on which we wish to approximate our function
X = np.linspace(range_min, range_max-range_min, n_grid)  # Optimisation grid
T = np.linspace(range_min, range_max-range_min, n_grid)  # Optimisation grid

In [ ]:
# Differential equation's exact solution - for comparison
def u(x,t):
    return x**2+t**2+x*t

In [ ]:
# Parameters of the quantum machine learning procedure
N = nphotons              # Number of photons
m = nphotons              # Number of modes
eta = 5                   # weight granted to the initial condition
a = 200                   # Approximate boundaries of the interval that the image of the trial function can cover
fock_dim = comb(N + m - 1, N)
# lambda coefficients for all the possible outputs
lambda_random = 2 * a * np.random.rand(fock_dim) - a
# dx serves for the numerical differentiation of f
dx = (range_max-range_min) / (n_grid - 1)

In [ ]:
# Input state with N photons and m modes
input_state = pcvl.BasicState([1]*N+[0]*(m-N))
print(input_state)

In [ ]:
"Haar unitary parameters"
# number of parameters used for the two universal interferometers (2*m**2 per interferometer)
parameters = np.random.normal(size=4*m**2)

px = pcvl.P("px")
pt = pcvl.P("pt")
c = pcvl.Unitary(pcvl.Matrix.random_unitary(m, parameters[:2 * m ** 2]), name="W1")\
     // (0, pcvl.PS(px))\
     // (1, pcvl.PS(pt))\
     // pcvl.Unitary(pcvl.Matrix.random_unitary(m, parameters[2 * m ** 2:]), name="W2")

simulator_backend = pcvl.BackendFactory().get_backend("SLOS")
s1 = simulator_backend(pcvl.Matrix.random_unitary(m))
s1.compile(input_state)

pcvl.pdisplay(c)

In [ ]:
# Q_U_xt = np.zeros(shape=(len(X),len(T)))

# for i, row in enumerate(Q_U_xt):
#     for j, element in enumerate(row):
#         Q_U_xt[i,j]=1
#         print(f"Element at ({i}, {j}): {element}")
# Q_U_xt

# Q_U_xt = np.zeros(shape=(len(X),len(T)))

# for i, row in enumerate(Q_U_xt):
#     for j, element in enumerate(row):
#         # print(f"Element at ({i}, {j}): {element}")
#         Q_U_xt[i,j] = (X[i]+T[j])
# Q_U_xt
    

In [ ]:
# A = np.linspace(0, 5, 6)
# B = np.linspace(0, 4, 6)

# A_mesh, B_mesh = np.meshgrid(A, B)
# print(A_mesh)
# print(B_mesh)

In [ ]:
# Y= np.array([1,4,3,4,8,6,7])
# print(Y[2:])
# print(Y[:-2])
# print(Y[2:]-Y[:-2])

# Y_prime = (Y[2:] - Y[:-2])/(2*dx)
# Y_prime

In [ ]:
# Y= np.array([[1,4,3,5],[0,1,3,7],[2,8,1,1],[2,8,1,1]])
Y= np.array([[1,4,3,5],[0,1,3,7],[0,1,0,0]])
# # print(Y)
# Col1= Y[:,0]
# # print(Col1)
# # print(Col1[2:]-Col1[:-2])

# row1= Y[0,:].copy()
# row1[0]=2
# print(row1)
print(Y)
print(np.sum(Y))
# print(row1[2:]-row1[:-2])



In [ ]:
def computation(params):
    global current_loss
    global computation_count
    "compute the loss function of a given differential equation in order for it to be optimized"
    computation_count += 1

    current_loss = 0

    coefs = lambda_random  # coefficients of the M observable
    # initial condition with the two universal interferometers and the phase shift in the middle
    U_1 = pcvl.Matrix.random_unitary(m, params[:2 * m ** 2])
    U_2 = pcvl.Matrix.random_unitary(m, params[2 * m ** 2:])

    px = pcvl.P("x")
    pt = pcvl.P("t")
    c = pcvl.Unitary(U_2) // (0, pcvl.PS(px))// (1, pcvl.PS(pt))  // pcvl.Unitary(U_1)


    loss_I1 = 0
    loss_B1 = 0
    loss_B2 = 0
    loss_B3 = 0


    #PDE
    Q_U_xt = np.zeros(shape=(len(X),len(T)))

    for i in range(len(X)):
        for j in range(len(T)):
            # print(f"Element at ({i}, {j}): {element}")
            px.set_value(np.pi * X[i])
            pt.set_value(np.pi * T[j])
            s1.U = c.compute_unitary(use_symbolic=False)
            Q_U_xt[i,j]=np.sum(np.multiply(s1.all_prob(input_state), coefs))
    # print(Q_U_xt)

    Q_U_prime_x = np.zeros(shape=(len(X),len(T)))
    # print(Q_U_prime_x)
    for j in range(len(T)):
        col_t_j = Q_U_xt[:,j]
        Q_U_prime_x[1:-1,j] = col_t_j[2:]-col_t_j[:-2]
    # print(Q_U_prime_x)
    

    Q_U_prime_t = np.zeros(shape=(len(X),len(T)))
    for i in range(len(X)):
        row_x_i = Q_U_xt[i,:]
        # print(row_x_i)
        Q_U_prime_t[i,1:-1] = row_x_i[2:]-row_x_i[:-2]
        # print(Q_U_prime_x[i,1:-1])
    # print(Q_U_prime_t)

    # Loss interiors
    x_mesh, t_mesh = np.meshgrid(X, T)
    loss_I1 = np.sum(F(Q_U_prime_x,Q_U_prime_t,x_mesh,t_mesh)**2)


    # Loss Boundary
    #U_0t
    Q_U_0t = Q_U_xt[0,:]
    loss_B1 += np.sum(B1(Q_U_0t,T)**2)
    
    #U_x0
    Q_U_x0 = Q_U_xt[:,0]
    loss_B2 += np.sum(B2(Q_U_x0,X)**2)  

    #U_00
    Q_U_00 = Q_U_xt[0,0]
    loss_B3 += np.sum(Q_U_00**2)  

    current_loss = loss_B1+loss_B2+loss_B3+loss_I1

    # print(current_loss)
    current_loss = current_loss / len(X)
    return current_loss

In [ ]:
# def computation(params):
#     global current_loss
#     global computation_count
#     "compute the loss function of a given differential equation in order for it to be optimized"
#     computation_count += 1
#     f_theta_0 = 0  # boundary condition
#     f_theta_1 = 0  # boundary condition

#     coefs = lambda_random  # coefficients of the M observable
#     # initial condition with the two universal interferometers and the phase shift in the middle
#     U_1 = pcvl.Matrix.random_unitary(m, params[:2 * m ** 2])
#     U_2 = pcvl.Matrix.random_unitary(m, params[2 * m ** 2:])

#     px = pcvl.P("x")
#     c = pcvl.Unitary(U_2) // (0, pcvl.PS(px)) // pcvl.Unitary(U_1)

#     px.set_value(np.pi * x_0)
#     U = c.compute_unitary(use_symbolic=False)
#     s1.U = U
#     f_theta_0 = np.sum(np.multiply(s1.all_prob(input_state), coefs))


#     px.set_value(np.pi * 1)
#     UU = c.compute_unitary(use_symbolic=False)
#     s1.U = UU
#     f_theta_1 = np.sum(np.multiply(s1.all_prob(input_state), coefs))

#     # boundary condition given a weight eta
#     loss = eta * (f_theta_0 - f_0) ** 2 * len(X)
#     loss += eta * (f_theta_1 - np.sin(1)) ** 2 * len(X)
#     # loss=0


#     # Y[0] is before the domain we are interested in (used for differentiation), x_0 is at Y[1]
#     Y = np.zeros(n_grid + 2)

#     # x_0 is at the beginning of the domain, already calculated
#     Y[1] = f_theta_0

#     px.set_value(np.pi * (range_min - dx))
#     s1.U = c.compute_unitary(use_symbolic=False)
#     Y[0] = np.sum(np.multiply(s1.all_prob(input_state), coefs))


#     for i in range(1, n_grid):
#         x = X[i]
#         px.set_value(np.pi * x)
#         s1.U = c.compute_unitary(use_symbolic=False)
#         Y[i + 1] = np.sum(np.multiply(s1.all_prob(input_state), coefs))

#     px.set_value(np.pi * (range_max + dx))
#     s1.U = c.compute_unitary(use_symbolic=False)
#     Y[n_grid + 1] = np.sum(np.multiply(s1.all_prob(input_state), coefs))


#     # Differentiation
#     Y_prime = (Y[2:] - Y[:-2])/(2*dx)

#     Y_zegond = (Y_prime[2:] - Y_prime[:-2])/(2*dx)

#     loss += np.sum((F(Y_zegond,Y_prime, Y[2:-2], X[1:-1]))**2)

#     current_loss = loss / len(X)
#     return current_loss

In [ ]:
def callbackF(parameters):
    """callback function called by scipy.optimize.minimize allowing to monitor progress"""
    global current_loss
    global computation_count
    global loss_evolution
    global start_time
    now = time.time()
    pbar.set_description("M= %d Loss: %0.5f #computations: %d elapsed: %0.5f" %
                         (m, current_loss, computation_count, now-start_time))
    pbar.update(1)
    loss_evolution.append((current_loss, now-start_time))
    computation_count = 0
    start_time = now

In [ ]:
computation_count = 0
current_loss = 0
start_time = time.time()
loss_evolution = []

pbar = tqdm.tqdm()
res = minimize(computation, parameters, callback=callbackF, method='BFGS', options={'gtol': 1E-2})

In [ ]:
print("Unitary parameters", res.x)
np.save(file=str(np.random.random())+'('+str(nphotons)+')'+'_params1(prime).ipy',arr=res.x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the range of x and t values
x = np.linspace(0, 0.5, 100)  # Range of x values
t = np.linspace(0, 0.5, 100)  # Range of t values

# Create a meshgrid of x and t values
XX, TT = np.meshgrid(x, t)

XX = x 
TT = t

# Compute the values of U(x, t) for the first code
# U1 = XX**2 + TT**2 + XX*TT
# U1 = u(XX,TT)
U1 = np.zeros((len(XX), len(TT)))
for i in range(len(XX)):
    for j in range(len(TT)):
        U1[i,j]=u(XX[i],TT[j])



# Create a 3D plot for the first code
fig = plt.figure()
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(XX, TT, U1, cmap='viridis')
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('U(x, t)')
ax1.set_title('Plot of U(x, t) = x^2 + t^2 + xt')

# Compute the values of U(x, t) for the second code
# Replace the code below with the actual calculation from the second code snippet
# predict
U2 = np.zeros((len(XX), len(TT)))
# print(len(TT))
U_1 = pcvl.Matrix.random_unitary(m, res.x[:2 * m ** 2])
U_2 = pcvl.Matrix.random_unitary(m, res.x[2 * m ** 2:])
px = pcvl.P("x")
pt = pcvl.P("t")
c = pcvl.Unitary(U_2) // (0, pcvl.PS(px)) // (1, pcvl.PS(pt))// pcvl.Unitary(U_1)


for i in range(len(XX)):
    for j in range(len(TT)):
        # print(X[i])
        # print(T[j]).
        # print(XX)
        px.set_value(np.pi * XX[i])
        pt.set_value(np.pi * TT[j])
        U = c.compute_unitary(use_symbolic=False)
        s1.U = U
        f_theta = np.sum(np.multiply(s1.all_prob(input_state), lambda_random))
        # print(f_theta)
        # print(U1[i,j])
        # print('====================')
        U2[i,j]=f_theta

# Create a 3D plot for the second code
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(XX, TT, U2, cmap='viridis')
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_zlabel('U(x, t)')
ax2.set_title('Quantum Plot of U(x, t)')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D


# def plot_solution(m, N, X,T, optim_params, lambda_random):
 
#     # Y = []
#     U_1 = pcvl.Matrix.random_unitary(m, optim_params[:2 * m ** 2])
#     U_2 = pcvl.Matrix.random_unitary(m, optim_params[2 * m ** 2:])
#     px = pcvl.P("x")
#     pt = pcvl.P("t")
#     c = pcvl.Unitary(U_2) // (0, pcvl.PS(px)) // (1, pcvl.PS(pt))// pcvl.Unitary(U_1)

#     # Create a meshgrid of x and t values
#     X, T = np.meshgrid(X, T)
#     F = np.zeros((len(X), len(T)))
#     for i in range(len(X)):
#         for j in range(len(T)):
#             # print(X[i])
#             # print(T[j])
#             px.set_value(np.pi * X[i])
#             pt.set_value(np.pi * T[j])
#             U = c.compute_unitary(use_symbolic=False)
#             s1.U = U
#             f_theta = np.sum(np.multiply(s1.all_prob(input_state), lambda_random))
#             F[i,j]=f_theta
#             # Y.append(f_theta)
            
#     # Create a 3D plot
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.plot_surface(X, T, F, cmap='viridis')

#     # Set labels and title
#     ax.set_xlabel('x')
#     ax.set_ylabel('t')
#     ax.set_zlabel('U(x, t)')
#     ax.set_title('Approximation Plot of U(x, t)')
#     # plt.plot(X, Y, label="Approximation with {} photons".format(N))

In [ ]:
# X = np.linspace(range_min, range_max, 200)
# T = np.linspace(range_min, range_max, 200)


# # Change the plot size
# default_figsize = mpl.rcParamsDefault['figure.figsize']
# mpl.rcParams['figure.figsize'] = [2 * value for value in default_figsize]

# plot_solution(m, N, X,T, res.x, lambda_random)

# # Create a meshgrid of x and t values
# X, T = np.meshgrid(X, T)

# # Compute the values of U(x, t)
# U = X**2 + T**2 + X*T


# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(X, T, U, cmap='viridis')

# # Set labels and title
# ax.set_xlabel('x')
# ax.set_ylabel('t')
# ax.set_zlabel('U(x, t)')
# ax.set_title('Plot of U(x, t)')

# # Show the plot
# plt.show()



# # plt.plot(X, u(X), 'r', label='Analytical solution')
# # plt.legend()
# # plt.show()

In [ ]:

plt.plot([v[0] for v in loss_evolution])
plt.yscale("log")
plt.xlabel("Number of epochs")
plt.ylabel("Loss function value")